In [1]:
import numpy as np
import pandas as pd
import scipy
from matplotlib import pylab, gridspec, pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

*Сохраняем данные*

In [42]:
df = pd.read_csv("data.csv", index_col=False)

*Вид данных*

In [43]:
df.head()

,arr_line,dest_line,arr_station,dest_station
0,Сокольническая линия,Сокольническая линия,Черкизовская,Преображенская площадь
1,Сокольническая линия,Сокольническая линия,Преображенская площадь,Сокольники
2,Сокольническая линия,Сокольническая линия,Сокольники,Красносельская
3,Сокольническая линия,Сокольническая линия,Красносельская,Комсомольская
4,Сокольническая линия,Сокольническая линия,Комсомольская,Красные ворота


In [44]:
df["arr_line"] = " " + df["arr_line"]

*Преобразуем в однородные данные*

In [7]:
new_df = pd.DataFrame(columns=['from', 'to'])

In [8]:
new_df['from'] = df['arr_line'] + df['arr_station']
new_df['to'] = df['dest_line'] + df['dest_station']
new_df.head()

,from,to
0,Сокольническая линия Черкизовская,Сокольническая линия Преображенская площадь
1,Сокольническая линия Преображенская площадь,Сокольническая линия Сокольники
2,Сокольническая линия Сокольники,Сокольническая линия Красносельская
3,Сокольническая линия Красносельская,Сокольническая линия Комсомольская
4,Сокольническая линия Комсомольская,Сокольническая линия Красные ворота


In [9]:
stations = np.unique([*new_df['from'], *new_df['to']])
dim = stations.shape[0]
adj_matrix = np.zeros([dim, dim], dtype='int')
adj_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
dict_stations = dict(zip(stations, range(stations.shape[0]))) 
rev_dict_stations = dict(zip(range(stations.shape[0]), stations)) 

In [11]:
df_num = new_df.copy()

In [12]:
df_num['from'] = df_num['from'].map(dict_stations)
df_num['to'] = df_num['to'].map(dict_stations)

In [13]:
df_num.head()

,from,to
0,164,158
1,158,160
2,160,152
3,152,151
4,151,153


In [14]:
data_l_in = list(df_num['from'])
data_l_out = list(df_num['to'])
for i in range(len(data_l_in)):
    adj_matrix[data_l_out[i]][data_l_in[i]] = adj_matrix[data_l_in[i]][data_l_out[i]] = 1

In [15]:
print(adj_matrix[2])

[0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [16]:
adj_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
def neighbors(station: int) -> list:
    list_neighbors = []
    for i in range(len(adj_matrix[station])):
        if adj_matrix[station][i] == 1:
            list_neighbors.append(rev_dict_stations[i])
    return list_neighbors
neighbors(0)

[' Арбатско-Покровская линия Площадь Революции',
 ' Арбатско-Покровская линия Смоленская',
 ' Серпуховско-Тимирязевская линия Боровицкая',
 ' Сокольническая линия Библиотека имени Ленина',
 ' Филевская линия Александровский сад']

In [18]:
# def dijkstra(graph: 'np.ndarray', s:int, t:int)->'np.ndarray':
#     '''finding the shortest paht between s-vertex & t-vertex'''
#     size = len(graph[0])
#     path = []
#     dist = [np.inf for _ in range(size)]
#     u = [False for _ in range(size)]
    
#     dist[s] = 0
    
#     for i in range(size):
# #         cur = 0
#         min_ = np.inf
#         for j in range(size):
#             if u[j] == False and dist[j] <= min_:
#                 min_ = dist[j]
#                 cur = j
        
#         u[cur] = True
        
#         for j in range(size):
#             if graph[cur][j] > 0 and u[j] == False:
#                 if dist[cur] + graph[cur][j] < dist[j]:
#                     dist[j] = dist[cur] + graph[cur][j]
    
# #     print(dist)
#     return dist

In [19]:
class Priority_queue(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.queue = sorted(self, key=lambda x: self[x])
    def __setitem__(self, key, val):
        dict.__setitem__(self, key, val)
        if not self.queue:
            self.queue.append(key)
        else:
            i = 0
            while i < len(self.queue) and val >= self[self.queue[i]]:
                i += 1
            self.queue.insert(i, key)
    def clear(self):
        dict.clear(self)
        self.queue.clear()
    def pop(self):
        key = self.queue[0]
        value = self[key]
        dict.pop(self, key)
        self.queue.pop(0)
        return key, value

In [20]:
class Graph:
    open_set = Priority_queue()
    cost_map = {}
    def __init__(self, matrix: "np.darray"):
        self.matrix = matrix
        self.dim = matrix.shape[0]
        self.distance_matrix = np.zeros(matrix.shape, dtype='int')
    
    def _neighbors(self, vertex: int) -> list:
        list_neighbors = []
        for i in range(len(self.matrix[vertex])):
            if self.matrix[vertex][i] == 1:
                list_neighbors.append(i)
        return list_neighbors
    
    def _update_open_set(self, vertex: int, cost: int):
        neighbors = self._neighbors(vertex)
        for i in neighbors:
            if i in self.cost_map:
                continue
            path_cost = self.matrix[vertex][i] + cost
            if i not in self.open_set:
                self.open_set[i] = path_cost
            else:
                if self.open_set[i] > path_cost:
                    self.open_set[i] = path_cost
        
    def _short_path(self, src: int, dst: int):
        self.cost_map.clear()
        self.open_set.clear()
        self._update_open_set(src, 0)
        while self.open_set:
            min_vertex, min_cost = self.open_set.pop()
            self.cost_map[min_vertex] = min_cost
            if min_vertex == dst:
                return
            self._update_open_set(min_vertex, min_cost)
            
    def _path_size(self, src: int, dst: int) -> int:
        if src == dst:
            return 0
        self._short_path(src, dst)
        return self.cost_map[dst]
    
    def distances(self) -> "np.darray":
        for i in range(self.dim):
            for j in range(i+1, self.dim):
                self.distance_matrix[i][j] = self.distance_matrix[j][i] = graph._path_size(i, j)
        return self.distance_matrix


In [21]:
%%time
# лучше не запускать
graph = Graph(adj_matrix)
distances = graph.distances()
distances

CPU times: user 4min 23s, sys: 1.25 s, total: 4min 24s
Wall time: 4min 28s


array([[ 0,  3, 10, ...,  4,  6,  8],
       [ 3,  0, 13, ...,  7,  9, 11],
       [10, 13,  0, ..., 10, 10,  8],
       ...,
       [ 4,  7, 10, ...,  0,  2,  4],
       [ 6,  9, 10, ...,  2,  0,  2],
       [ 8, 11,  8, ...,  4,  2,  0]])

In [22]:
df_dist = pd.DataFrame(distances)
df_dist.to_csv("distance_data.csv")
df_dist.head()

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,201
0,0,3,10,7,2,7,5,2,11,6,...,4,3,6,5,5,7,3,4,6,8
1,3,0,13,4,5,10,8,1,14,9,...,7,6,9,8,8,10,6,7,9,11
2,10,13,0,17,8,3,5,12,1,4,...,10,9,6,11,11,7,10,10,10,8
3,7,4,17,0,9,14,12,5,18,13,...,11,10,13,12,12,14,10,11,13,15
4,2,5,8,9,0,5,3,4,9,4,...,2,1,4,3,3,5,2,2,4,6


In [23]:
df_names = pd.DataFrame(stations, columns=['name'])
df_names['coffee'] = 0
df_names['tea'] = 0

In [24]:
tea_map = {' Замоскворецкая линия Царицыно': 2, ' Таганско-Краснопресненская линия Текстильщики': 1, '  Замоскворецкая линия Войковская': 2, ' Калужско-Рижская линия Бабушкинская': 1, ' Серпуховско-Тимирязевская линия Отрадное': 1, ' Замоскворецкая линия Орехово': 1, ' Замоскворецкая линия Ховрино': 1}
coffee_map = {' Калужско-Рижская линия Академическая': 1, ' Серпуховско-Тимирязевская линия Тимирязевская': 1, ' Люблинская линия Окружная': 1, ' Арбатско-Покровская линия Молодёжная': 1, '  Люблинская линия Бутырская': 1, ' Кольцевая линия Комсомольская': 1, ' Замоскворецкая линия Речной вокзал': 1, ' Замоскворецкая линия Войковская': 2, ' Замоскворецкая линия Сокол': 1}




In [25]:
print(coffee_map, tea_map, sep= '\n\n')

{' Калужско-Рижская линия Академическая': 1, ' Серпуховско-Тимирязевская линия Тимирязевская': 1, ' Люблинская линия Окружная': 1, ' Арбатско-Покровская линия Молодёжная': 1, '  Люблинская линия Бутырская': 1, ' Кольцевая линия Комсомольская': 1, ' Замоскворецкая линия Речной вокзал': 1, ' Замоскворецкая линия Войковская': 2, ' Замоскворецкая линия Сокол': 1}

{' Замоскворецкая линия Царицыно': 2, ' Таганско-Краснопресненская линия Текстильщики': 1, '  Замоскворецкая линия Войковская': 2, ' Калужско-Рижская линия Бабушкинская': 1, ' Серпуховско-Тимирязевская линия Отрадное': 1, ' Замоскворецкая линия Орехово': 1, ' Замоскворецкая линия Ховрино': 1}


In [26]:
df_names.head(10)

,name,coffee,tea
0,Арбатско-Покровская линия Арбатская,0,0
1,Арбатско-Покровская линия Бауманская,0,0
2,Арбатско-Покровская линия Волоколамская,0,0
3,Арбатско-Покровская линия Измайловская,0,0
4,Арбатско-Покровская линия Киевская,0,0
5,Арбатско-Покровская линия Крылатское,0,0
6,Арбатско-Покровская линия Кунцевская,0,0
7,Арбатско-Покровская линия Курская,0,0
8,Арбатско-Покровская линия Митино,0,0
9,Арбатско-Покровская линия Молодёжная,0,0


In [27]:
for i, row in df_names[df_names['name'].isin(coffee_map.keys())].iterrows():
    df_names.at[i, 'coffee'] = coffee_map[df_names.at[i, 'name']]
for i, row in df_names[df_names['name'].isin(tea_map.keys())].iterrows():
    df_names.at[i, 'tea'] = tea_map[df_names.at[i, 'name']]


In [28]:
df_names[df_names['coffee'] == 1]

,name,coffee,tea
9,Арбатско-Покровская линия Молодёжная,1,0
46,Замоскворецкая линия Речной вокзал,1,0
47,Замоскворецкая линия Сокол,1,0
62,Калужско-Рижская линия Академическая,1,0
92,Кольцевая линия Комсомольская,1,0
114,Люблинская линия Окружная,1,0
142,Серпуховско-Тимирязевская линия Тимирязевская,1,0


In [29]:
df_names[df_names['tea'] == 1]

,name,coffee,tea
44,Замоскворецкая линия Орехово,0,1
50,Замоскворецкая линия Ховрино,0,1
64,Калужско-Рижская линия Бабушкинская,0,1
135,Серпуховско-Тимирязевская линия Отрадное,0,1
185,Таганско-Краснопресненская линия Текстильщики,0,1


In [30]:
df_dist['coffee'] = df_names['coffee']
df_dist['tea'] = df_names['tea']

In [31]:
df_dist[df_dist['coffee'] != 0][0] 

9       6
35      9
46     11
47      8
62      8
92      4
114    10
142     7
Name: 0, dtype: int64

In [32]:
def guess_class(x: int, k: int):
    "определение класса для станции x по k соседям"
    return

In [33]:
def guess_class(x,y,k):
    '''
        Функция отрисовки окрестности точки (x,y) и функция поиска расстояния между точками b,c на плоскости
    '''
    def draw_points(x,y):
        plt.figure(figsize=(5,5))
        plt.grid(ls='--')
        plt.scatter(a,blues,c='b')
        plt.scatter(a,reds,c='r')
        plt.scatter(x,y,c='g')
        plt.xlim(x-7, x+7)
        plt.ylim(y-7, y+7)
    draw_points(x,y)
    
    def distance_2d(b,c):
        return np.sqrt((b[0]-c[0])**2+(b[1]-c[1])**2)
    '''
        Функция подсчета расстояний между ближайшими k точками
    '''
    def calculate_k_distances(class_points,l):
        distances = []
        labels = []
        for point in class_points:
            # считаем расстояние от точки до точки класса
            distances.append(distance_2d(point,[x,y]))
        distances = np.sort(distances)[::-1]
        return [(x,l) for x in distances[:k]]  
             
    b = np.array(calculate_k_distances(blue_pairs,'b')) # расстояния до ближайших k соседей класса blue
    r = np.array(calculate_k_distances(red_pairs,'r')) # расстояния до ближайших k соседей класса red
    
    print('Ближайшие {} синих соседей'.format(k),b)
    print('Ближайшие {} красных соседей'.format(k),r)
    nearest_k_dists = pd.DataFrame(np.concatenate((b,r)),
                                   columns=['distance', 'label']).sort_values(by='distance',
                                                                              ascending=False)[::-1][:k]
    print('======================================================')
    print(nearest_k_dists)
    r, b =0, 0 
    for i in range(2*k):
        try:
            if nearest_k_dists['label'][i] == 'r':
                r += 1
            else:
                b += 1
        except:
            continue
    if r > b :
        print('red')
    else:
        print('blue')